## Import Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
# Setting a seed ensures that the random numbers generated by NumPy will be the same each time the code is run
np.random.seed(1)

# Sets the behavior to ignore warnings specifically related to 
# divide-by-zero (divide='ignore') and invalid value 
# computations (invalid='ignore').
np.seterr(divide='ignore', invalid='ignore')

# Adjusts the pandas library's behavior when it encounters 
# chained assignment warnings. Chained assignment occurs when 
# multiple operations are performed successively on a DataFrame 
# without explicit copy or assignment.
pd.options.mode.chained_assignment = None

### Datasets:  
- **Telco Customer Churn:** https://www.kaggle.com/blastchar/telco-customer-churn  

### Telco Customer Churn Dataset Preprocessing

In [ ]:
def telco_customer_churn_dataset_preprocessing():
    # Import the required file
    df = pd.read_csv("../Datasets/WA_Fn-UseC_-Telco-Customer-Churn.csv")

    # Dropping unnecessary columns
    # `axis=1` specifies that columns are to be dropped.
    # `inplace=True` modifies the DataFrame in place, 
    # without creating a new DataFrame.
    df.drop(['customerID'], inplace=True, axis=1)

    # Modify specific column values
    no_internet_cols = ['OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
	                'TechSupport', 'StreamingTV', 'StreamingMovies']
    
    for col in no_internet_cols:
        df[col].replace({'No internet service': 'No'}, inplace=True)
	
    df['MultipleLines'].replace({'No phone service': 'No'}, inplace=True)

    # Replacing Binary column values with 0 & 1
    yes_no_cols = ['Partner', 'Dependents', 'PhoneService', 'MultipleLines'] \
	               + no_internet_cols + ['PaperlessBilling', 'Churn']

    for col in yes_no_cols:
        df[col].replace({'Yes': 1, 'No': 0}, inplace=True)
        
    df['gender'].replace({'Male': 1, 'Female': 0}, inplace=True)
    
    # One-Hot Encoding for Multi-value features
    df = pd.get_dummies(
        df, 
        columns=[
            'InternetService', 'Contract', 'PaymentMethod'
        ]
    )
    
    # Create an imputer (numeric_imputer) to fill missing numeric values with the mean.
    numeric_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
    
    df['TotalCharges'].replace({' ': np.nan}, inplace=True)
    df['TotalCharges'] = numeric_imputer.fit_transform(df[['TotalCharges']])
    
    # Separating target column from data frame
    data_frame_target = df['Churn']
    df.drop(['Churn'], inplace=True, axis=1)
    
    # Standardizing specific columns in data frame
    standard_scaler = StandardScaler()
    
    for col in ['tenure', 'MonthlyCharges', 'TotalCharges']:
        df[col] = standard_scaler.fit_transform(df[[col]])
    
    # Returns the processed data as NumPy arrays (features and target).
    return df.to_numpy(), data_frame_target.to_numpy()

### Datasets:  
- **Adult Salary Scale:** https://archive.ics.uci.edu/ml/datasets/adult  

### Adult Salary Scale Dataset Preprocessing

In [ ]:
column_labels = ['age', 'workclass', 'fnlwgt', 'education', 
                'education-num', 'marital-status', 'occupation',
                'relationship', 'race', 'sex', 'capital-gain', 
                'capital-loss', 'hours-per-week',
                'native-country', 'salary']

numeric_colms = ['age', 'fnlwgt', 'education-num', 'capital-gain', 
                'capital-loss', 'hours-per-week']


def adult_dataset_preprocessing():
    df_train = pd.read_csv("../Datasets/adult/adult.data", header=None, names=column_labels)
    df_test = pd.read_csv("../Datasets/adult/adult.test", header=None, skiprows=1, names=column_labelss)
    
    # df_train.columns = column_labels
    # df_test.columns = column_labels
    
    # Removes leading and trailing whitespaces from each string 
    # element within the column
    for col in column_labels:
        df_train[col] = df_train[col].astype(str).str.strip()
        df_test[col] = df_test[col].astype(str).str.strip()
        
    # Modify values in specific columns
    df_train['salary'].replace({'>50K': 1, '<=50K': 0}, inplace=True)
    df_test['salary'].replace({'>50K.': 1, '<=50K.': 0}, inplace=True)
    
    df_train['sex'].replace({'Male': 1, 'Female': 0}, inplace=True)
    df_test['sex'].replace({'Male': 1, 'Female': 0}, inplace=True)
    
    # Create an imputer (numeric_imputer) to fill missing numeric values with the mean.
    numeric_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

    for col in numeric_colms:
        df_train[col].replace({'?': np.nan}, inplace=True)
        df_train[col] = numeric_imputer.fit_transform(df_train[[col]])
        df_test[col].replace({'?': np.nan}, inplace=True)
        df_test[col] = numeric_imputer.fit_transform(df_test[[col]])
    
    
    # Concatenating train_data_frame and test_data_frame
    df_train['is_train'] = 1
    df_test['is_train'] = 0
    data_frame = pd.concat([df_train, df_test], ignore_index=True)
    
    # Create an imputer (numeric_imputer) to fill missing numeric values with the most_frequent.
    categorical_imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    
    for col in ['workclass', 'occupation', 'native-country']:
        data_frame[col].replace({'?': np.nan}, inplace=True)
        data_frame[col] = categorical_imputer.fit_transform(data_frame[[col]])
    
    # One-Hot Encoding for Multi-value features
    data_frame = pd.get_dummies(
        data_frame, 
        columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'native-country']
    )
    
    # Remove column with only a single value column
    data_frame.drop(['native-country_Holand-Netherlands'], inplace=True, axis=1)
    
    # Separating df_train and df_test from data_frame
    df_train = data_frame[data_frame['is_train'] == 1]
    df_test = data_frame[data_frame['is_train'] == 0]
    
    df_train.drop(['is_train'], inplace=True, axis=1)
    df_test.drop(['is_train'], inplace=True, axis=1)
    
    # Separating target column from data frame
    df_train_target = df_train['salary']
    df_train.drop(['salary'], inplace=True, axis=1)
    
    df_test_target = df_test['salary']
    df_test.drop(['salary'], inplace=True, axis=1)
    
    # Standardizing specific columns in data frame
    standard_scaler = StandardScaler()
    
    for col in ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']:
        df_train[col] = standard_scaler.fit_transform(df_train[[col]])
        df_test[col] = standard_scaler.fit_transform(df_test[[col]])   
    
    return (
        df_train.to_numpy(), 
        df_train_target.to_numpy(), 
        df_test.to_numpy(), 
        df_test_target.to_numpy()
    )

### Datasets:  
- **Credit Card Fraud Detection:** https://www.kaggle.com/mlg-ulb/creditcardfraud

### Credit Card Fraud Detection Dataset Preprocessing

In [ ]:
def credit_card_fraud_detection_dataset_preprocessing(use_smaller_subset=False):
    df = pd.read_csv('../Datasets/creditcard.csv')
    
    # Separating data samples based on value in 'Class' column
    df_0 = df[df['Class'] == 0]
    df_1 = df[df['Class'] == 1]
    
    # Splitting data frame 0(df_0) and data frame 1(df_1) into 
    # training and testing sets
    train_df_0 = df_0.sample(frac=0.8, random_state=1)
    test_df_0 = df_0.drop(train_df_0.index)
    
    if use_smaller_subset:
        train_df_0 = train_df_0.sample(n=16000, random_state=1)
        test_df_0= test_df_0.sample(n=4000, random_state=1)
    
    train_df_1 = df_1.sample(frac=0.8, random_state=1)
    test_df_1 = df_1.drop(train_df_1.index)
    
    # Concatenating train_df_0 and train_df_1, test_df_0 and test_df_1
    df_train = pd.concat([train_df_0, train_df_1], ignore_index=True).sample(frac=1, random_state=1)
    df_test = pd.concat([test_df_0, test_df_1], ignore_index=True).sample(frac=1, random_state=1)
    
    # Separating target columns from data frames
    df_train_target = df_train['Class']
    df_train.drop(['Class'], inplace=True, axis=1)
    
    df_test_target = df_test['Class']
    df_test.drop(['Class'], inplace=True, axis=1)
    
    # Standardizing specific columns in data frames
    standard_scaler = StandardScaler()
    
    for col in list(df_train.columns):
        df_train[col] = standard_scaler.fit_transform(df_train[[col]])
        
    for col in list(df_test.columns):
        df_test[col] = standard_scaler.fit_transform(df_test[[col]])
    
    return (
        df_train.to_numpy(), 
        df_train_target.to_numpy(), 
        df_test.to_numpy(), 
        df_test_target.to_numpy()
    )